***GENERATED CODE FOR dataprepautoml PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
from hdfs3 import HDFileSystem
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs = HDFileSystem(
            host=eval(config)['host'], port=eval(config)['port'])
        with hdfs.open(eval(config)['url']) as f:
            df = pd.read_csv(f, error_bad_lines=False)
        df = spark.createDataFrame(dfPd)
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***OPERATION FUNCTIONS***

In [ ]:
import pyspark
import json
from dask.dataframe import from_pandas


def runDataCleansing(sparkDf, spark, config):
    configObj = json.loads(config)
    sparkDf.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = from_pandas((sparkDf.toPandas()), npartitions=5)
    functionList = configObj['functionsApplied']
    Data_Cleansing_Methods = {"replaceBy": replaceValues,
                              "formula": calculateFormula,
                              "aggregate": aggregation,
                              "converttostringtype": changeToString,
                              "editname": renameColumns}
    for function in functionList:
        function['functionName']
        df = Data_Cleansing_Methods[function['functionName']](df, function['functionsData'],
                                                              function['applyOn'])
    sparkDf = spark.createDataFrame(df.compute())

    display(sparkDf.limit(2).toPandas())
    return sparkDf


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.sql.functions import col, when
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def replaceByMean(self, feature, df, mean_=-1):

        meanValue = df.select(mean(col(feature.name)).alias(
            'mean')).collect()[0]["mean"]
        df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                      meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        maxValue = df.select(max(col(feature.name)).alias('max')).collect()[
            0]["max"]
        df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        minValue = df.select(min(col(feature.name)).alias('min')).collect()[
            0]["min"]
        df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        stddevValue = df.select(stddev(col(feature.name)).alias(
            'stddev')).collect()[0]["stddev"]
        df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        fillValue = df.where(col(feature.name).isNotNull()
                             ).head(1)[0][feature.name]
        df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
from sklearn.model_selection import train_test_split
from tpot import TPOTRegressor
import pyspark


def functionRegression(sparkDF, listOfFeatures, label):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = sparkDF.toPandas()
    X = (df.drop(label, axis=1))[listOfFeatures].values
    y = df[label].values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=1, test_size=0.1)
    tpotModel = TPOTRegressor(verbosity=3, generations=10, max_time_mins=5,
                              n_jobs=-1, random_state=25, population_size=15)
    tpotModel.fit(X_train, y_train)
    display(" Error rate of Model : %s" % tpotModel.score(X_test, y_test))
    data = {'model': tpotModel,
            'X_test': X_test,
            'y_test': y_test,
            'label': label,
            'columnNames': listOfFeatures}
    return data


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################from pyspark.sql import SparkSession
spark = SparkSession.builder.master('spark://0.0.0.0:0000').getOrCreate()
#%run dataprepautomlHooks.ipynb
try:
	#sourcePreExecutionHook()

	bostontrain = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/uploadedSourceFiles/BostonTrain.csv', 'filename': 'BostonTrain.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'is_header': 'Use Header Line', 'domain': 'http://172.17.0.7', 'port': '50070', 'dirPath': '/FileStore/platform', 'server_url': '/numtraPlatform/NumtraPlatformV2/uploads/platform/'}")
	#sourcePostExecutionHook(bostontrain)

except Exception as ex: 
	logging.error(ex)


***PERFORMING OPERATIONS***

In [ ]:
#%run dataprepautomlHooks.ipynb
try:
	#operationPreExecutionHook()

datapreparation = runDataCleansing(bostontrain,spark,json.dumps( {"url": "/FileStore/platform/uploadedSourceFiles/BostonTrain.csv", "source_attributes": {}, "DataPrepFile": "/FileStore/platform/uploadedSourceFiles/BostonTrain.csv", "data_source": "hdfs", "startListenerOnly": 1, "dateColumnNames": [], "FilePath": "/FileStore/platform/extra/63524bed96352a2c04c149251666338116/0part.csv", "requestRatio": 1.0, "totalRows": 506, "BasicStats": {"missingValues": 0.0, "numberOfColumns": 14, "numberOfRows": 506, "duplicateRowCount": 0, "stats": [{"column": "CRIM", "alias": "CRIM", "generated": 0, "type": "real", "max": 88.9762, "min": 0.00632, "mean": 3.613523557312254, "missing": 0.0, "stddev": 8.6, "outliers": [8.98296, 13.5222, 9.2323, 8.26725, 11.1081, 18.4982, 19.6091, 15.288, 9.82349, 23.6482, 17.8667, 88.9762, 15.8744, 9.18702, 20.0849, 16.8118, 24.3938, 22.5971, 14.3337, 11.5779, 8.64476, 13.3598, 8.71675, 38.3518, 9.91655, 25.0461, 14.2362, 9.59571, 24.8017, 41.5292, 67.9208, 20.7162, 11.9511, 14.4383, 51.1358, 14.0507, 18.811, 28.6558, 45.7461, 18.0846, 10.8342, 25.9406, 73.5341, 11.8123, 11.0874, 12.0482, 8.79212, 15.8603, 12.2472, 37.6619, 9.33889, 8.49213, 10.0623, 13.9134, 11.1604, 14.4208, 15.1772, 13.6781, 9.39063, 22.0511, 9.72418, 9.96654, 12.8023, 10.6718, 9.92485, 9.32909, 8.24809, 9.51363, 8.20058, 15.5757, 13.0751, 15.0234, 10.233, 14.3337], "validation": []}, {"column": "ZN", "alias": "ZN", "generated": 0, "type": "real", "max": 100.0, "min": 0.0, "mean": 11.363636363636363, "missing": 0.0, "stddev": 23.32, "outliers": [75.0, 75.0, 75.0, 90.0, 85.0, 100.0, 80.0, 80.0, 45.0, 45.0, 45.0, 45.0, 45.0, 45.0, 60.0, 60.0, 80.0, 80.0, 80.0, 80.0, 95.0, 95.0, 82.5, 82.5, 95.0, 95.0, 30.0, 30.0, 30.0, 30.0, 30.0, 30.0, 80.0, 80.0, 90.0, 40.0, 40.0, 40.0, 40.0, 40.0, 90.0, 90.0, 55.0, 80.0, 52.5, 52.5, 52.5, 80.0, 80.0, 80.0, 70.0, 70.0, 70.0, 34.0, 34.0, 34.0, 33.0, 33.0, 33.0, 33.0, 35.0, 35.0, 35.0, 55.0, 55.0, 85.0, 80.0, 40.0, 40.0, 60.0, 60.0, 90.0, 80.0, 80.0], "validation": []}, {"column": "INDUS", "alias": "INDUS", "generated": 0, "type": "real", "max": 27.74, "min": 0.46, "mean": 11.13677865612648, "missing": 0.0, "stddev": 6.86, "outliers": [], "validation": []}, {"column": "CHAS", "alias": "CHAS", "generated": 0, "type": "real", "max": 1.0, "min": 0.0, "mean": 0.0691699604743083, "missing": 0.0, "stddev": 0.25, "outliers": [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], "validation": []}, {"column": "NOX", "alias": "NOX", "generated": 0, "type": "real", "max": 0.871, "min": 0.385, "mean": 0.5546950592885376, "missing": 0.0, "stddev": 0.12, "outliers": [0.871, 0.871, 0.871, 0.871, 0.871, 0.871, 0.871, 0.871, 0.871, 0.871, 0.871, 0.871, 0.871, 0.871, 0.871, 0.871], "validation": []}, {"column": "RM", "alias": "RM", "generated": 0, "type": "real", "max": 8.78, "min": 3.561, "mean": 6.284634387351779, "missing": 0.0, "stddev": 0.7, "outliers": [8.069, 7.82, 4.903, 4.926, 7.802, 8.375, 7.929, 7.765, 7.831, 7.875, 7.61, 7.853, 8.034, 8.266, 8.725, 8.04, 7.686, 8.337, 8.247, 8.259, 8.704, 8.398, 8.297, 7.691, 7.82, 7.645, 7.923, 8.78, 3.561, 3.863, 4.906, 4.138, 4.368, 4.652, 4.88, 4.138, 4.628, 4.519], "validation": []}, {"column": "AGE", "alias": "AGE", "generated": 0, "type": "real", "max": 100.0, "min": 2.9, "mean": 68.57490118577076, "missing": 0.0, "stddev": 28.15, "outliers": [], "validation": []}, {"column": "DIS", "alias": "DIS", "generated": 0, "type": "real", "max": 12.1265, "min": 1.1296, "mean": 3.795042687747036, "missing": 0.0, "stddev": 2.11, "outliers": [9.1876, 9.2229, 9.2203, 9.2203, 9.0892, 10.7103, 10.7103, 12.1265, 10.5857, 10.5857], "validation": []}, {"column": "RAD", "alias": "RAD", "generated": 0, "type": "real", "max": 24.0, "min": 1.0, "mean": 9.549407114624506, "missing": 0.0, "stddev": 8.71, "outliers": [], "validation": []}, {"column": "TAX", "alias": "TAX", "generated": 0, "type": "real", "max": 711.0, "min": 187.0, "mean": 408.2371541501976, "missing": 0.0, "stddev": 168.54, "outliers": [], "validation": []}, {"column": "PTRATIO", "alias": "PTRATIO", "generated": 0, "type": "real", "max": 22.0, "min": 12.6, "mean": 18.455533596837945, "missing": 0.0, "stddev": 2.16, "outliers": [12.6, 12.6, 12.6, 13.0, 13.0, 13.0, 13.0, 13.0, 13.0, 13.0, 13.0, 13.0, 13.0, 13.0, 13.0, 13.6], "validation": []}, {"column": "B", "alias": "B", "generated": 0, "type": "real", "max": 396.9, "min": 0.32, "mean": 356.6740316205534, "missing": 0.0, "stddev": 91.29, "outliers": [288.99, 303.42, 306.38, 232.6, 248.31, 70.8, 344.91, 338.63, 262.76, 172.91, 169.27, 341.6, 343.28, 261.95, 321.02, 88.01, 88.63, 338.92, 240.16, 227.61, 297.09, 330.04, 292.29, 348.13, 348.93, 341.6, 316.03, 131.42, 347.88, 285.83, 338.16, 329.46, 332.09, 314.64, 179.36, 2.6, 35.05, 28.79, 210.97, 88.27, 27.25, 21.57, 127.36, 16.45, 48.45, 318.75, 319.98, 291.55, 2.52, 3.65, 7.68, 24.65, 18.82, 96.73, 60.72, 83.45, 81.33, 97.95, 100.19, 100.63, 109.85, 27.49, 9.32, 68.95, 240.52, 43.06, 318.01, 304.21, 0.32, 6.68, 50.92, 10.48, 3.5, 272.21, 255.23, 334.4, 22.01, 331.29, 302.76, 344.05, 318.43], "validation": []}, {"column": "LSTAT", "alias": "LSTAT", "generated": 0, "type": "real", "max": 37.97, "min": 1.73, "mean": 12.653063241106722, "missing": 0.0, "stddev": 7.14, "outliers": [29.93, 30.81, 34.41, 29.53, 29.55, 34.77, 37.97, 30.63, 30.81, 31.99, 30.62, 30.59, 29.97, 34.37, 36.98, 34.02, 29.68], "validation": []}, {"column": "MEDV", "alias": "MEDV", "generated": 0, "type": "real", "max": 50.0, "min": 5.0, "mean": 22.532806324110677, "missing": 0.0, "stddev": 9.2, "outliers": [36.2, 35.4, 38.7, 43.8, 41.3, 50.0, 50.0, 50.0, 50.0, 37.2, 39.8, 36.2, 37.9, 50.0, 37.0, 36.4, 50.0, 42.3, 48.5, 50.0, 44.8, 50.0, 37.6, 46.7, 41.7, 48.3, 42.8, 44.0, 50.0, 36.0, 43.1, 48.8, 36.5, 50.0, 43.5, 35.2, 35.1, 45.4, 35.4, 46.0, 50.0, 37.3, 36.1, 50.0, 50.0, 50.0, 50.0, 50.0, 5.0, 6.3, 5.6, 5.0, 7.0, 7.0], "validation": []}]}, "predictionPowerScore": [{"AGE": 1.0, "B": 0.0, "CHAS": 0.0, "CRIM": 0.1819418929, "DIS": 0.3654966571, "INDUS": 0.4743910252, "LSTAT": 0.031921721, "MEDV": 0.12910914, "NOX": 0.501954171, "PTRATIO": 0.3398705519, "RAD": 0.160634802, "RM": 0.0, "TAX": 0.4286750281, "ZN": 0.2250916338}, {"AGE": 0.0, "B": 1.0, "CHAS": 0.0, "CRIM": 0.0, "DIS": 0.0, "INDUS": 0.0, "LSTAT": 0.0, "MEDV": 0.0, "NOX": 0.1311605518, "PTRATIO": 0.0, "RAD": 0.0, "RM": 0.0, "TAX": 0.0, "ZN": 0.0}, {"AGE": 0.0, "B": 0.0, "CHAS": 1.0, "CRIM": 0.0, "DIS": 0.0, "INDUS": 0.0, "LSTAT": 0.0, "MEDV": 0.0, "NOX": 0.0, "PTRATIO": 0.0, "RAD": 0.0, "RM": 0.0, "TAX": 0.0, "ZN": 0.0}, {"AGE": 0.0, "B": 0.0, "CHAS": 0.0, "CRIM": 1.0, "DIS": 0.0, "INDUS": 0.3943513349, "LSTAT": 0.0, "MEDV": 0.0, "NOX": 0.4181366721, "PTRATIO": 0.3498679351, "RAD": 0.3439931322, "RM": 0.0, "TAX": 0.3892382262, "ZN": 0.0}, {"AGE": 0.2263840804, "B": 0.0, "CHAS": 0.0, "CRIM": 0.1098016755, "DIS": 1.0, "INDUS": 0.7192599348, "LSTAT": 0.0, "MEDV": 0.0, "NOX": 0.7685256976, "PTRATIO": 0.4735583747, "RAD": 0.2010614961, "RM": 0.0, "TAX": 0.6826358869, "ZN": 0.3440131232}, {"AGE": 0.2109846408, "B": 0.007361998, "CHAS": 0.0, "CRIM": 0.4290394368, "DIS": 0.4042352387, "INDUS": 1.0, "LSTAT": 0.0617684759, "MEDV": 0.2447842529, "NOX": 0.9094313843, "PTRATIO": 0.6640834779, "RAD": 0.4417218805, "RM": 0.0, "TAX": 0.8713858002, "ZN": 0.2662979852}, {"AGE": 0.062603761, "B": 0.0, "CHAS": 0.0, "CRIM": 0.0, "DIS": 0.0983268519, "INDUS": 0.3050010509, "LSTAT": 1.0, "MEDV": 0.336130308, "NOX": 0.3489632496, "PTRATIO": 0.1996542817, "RAD": 0.1740047184, "RM": 0.0872401293, "TAX": 0.2588577211, "ZN": 0.130676928}, {"AGE": 0.0, "B": 0.0, "CHAS": 0.0, "CRIM": 0.0, "DIS": 0.0, "INDUS": 0.2739922522, "LSTAT": 0.192382389, "MEDV": 1.0, "NOX": 0.3534215648, "PTRATIO": 0.1830365575, "RAD": 0.0822970281, "RM": 0.134605752, "TAX": 0.1831393252, "ZN": 0.0969028871}, {"AGE": 0.2214231031, "B": 0.0, "CHAS": 0.0, "CRIM": 0.3529646943, "DIS": 0.3969063132, "INDUS": 0.7475713158, "LSTAT": 0.0, "MEDV": 0.1373945132, "NOX": 1.0, "PTRATIO": 0.523485966, "RAD": 0.3203099503, "RM": 0.0, "TAX": 0.6772530451, "ZN": 0.2350807327}, {"AGE": 0.0, "B": 0.0, "CHAS": 0.0, "CRIM": 0.0673137855, "DIS": 0.0907940989, "INDUS": 0.9172705087, "LSTAT": 0.0, "MEDV": 0.0215388598, "NOX": 0.8518213838, "PTRATIO": 1.0, "RAD": 0.3315092837, "RM": 0.0, "TAX": 0.7655938121, "ZN": 0.1993584355}, {"AGE": 0.0, "B": 0.0, "CHAS": 0.0, "CRIM": 0.7525769844, "DIS": 0.0, "INDUS": 0.9783928152, "LSTAT": 0.0, "MEDV": 0.0667400174, "NOX": 0.9687788517, "PTRATIO": 0.8051119616, "RAD": 1.0, "RM": 0.0, "TAX": 0.9212987046, "ZN": 0.0}, {"AGE": 0.0, "B": 0.0, "CHAS": 0.0, "CRIM": 0.0, "DIS": 0.0, "INDUS": 0.1465495983, "LSTAT": 0.0, "MEDV": 0.1446528159, "NOX": 0.1310865934, "PTRATIO": 0.0976885308, "RAD": 0.005901464, "RM": 1.0, "TAX": 0.0524240323, "ZN": 0.0894627131}, {"AGE": 0.0, "B": 0.0, "CHAS": 0.0, "CRIM": 0.5597919875, "DIS": 0.1597987317, "INDUS": 0.96937539, "LSTAT": 0.0, "MEDV": 0.1921251742, "NOX": 0.9474391316, "PTRATIO": 0.7269644848, "RAD": 0.6943762355, "RM": 0.0, "TAX": 1.0, "ZN": 0.075200725}, {"AGE": 0.0, "B": 0.0, "CHAS": 0.0, "CRIM": 0.0, "DIS": 0.4205587427, "INDUS": 0.8555653668, "LSTAT": 0.0, "MEDV": 0.0, "NOX": 0.7167584404, "PTRATIO": 0.3032268596, "RAD": 0.0, "RM": 0.0, "TAX": 0.5283745907, "ZN": 1.0}], "HasBasicStats": 1, "functionsApplied": [], "functionChanges": [], "fileheader": [{"field": "CRIM", "alias": "CRIM", "generated": 0, "position": 1, "type": "real"}, {"field": "ZN", "alias": "ZN", "generated": 0, "position": 2, "type": "real"}, {"field": "INDUS", "alias": "INDUS", "generated": 0, "position": 3, "type": "real"}, {"field": "CHAS", "alias": "CHAS", "generated": 0, "position": 4, "type": "real"}, {"field": "NOX", "alias": "NOX", "generated": 0, "position": 5, "type": "real"}, {"field": "RM", "alias": "RM", "generated": 0, "position": 6, "type": "real"}, {"field": "AGE", "alias": "AGE", "generated": 0, "position": 7, "type": "real"}, {"field": "DIS", "alias": "DIS", "generated": 0, "position": 8, "type": "real"}, {"field": "RAD", "alias": "RAD", "generated": 0, "position": 9, "type": "real"}, {"field": "TAX", "alias": "TAX", "generated": 0, "position": 10, "type": "real"}, {"field": "PTRATIO", "alias": "PTRATIO", "generated": 0, "position": 11, "type": "real"}, {"field": "B", "alias": "B", "generated": 0, "position": 12, "type": "real"}, {"field": "LSTAT", "alias": "LSTAT", "generated": 0, "position": 13, "type": "real"}, {"field": "MEDV", "alias": "MEDV", "generated": 0, "position": 14, "type": "real"}]}))
	#operationPostExecutionHook(datapreparation)

except Exception as ex: 
	logging.error(ex)


***TRANSFORMING DATAFRAME***

In [ ]:
#%run dataprepautomlHooks.ipynb
try:
	#transformationPreExecutionHook()

	autofe = TransformationMain.run(datapreparation,json.dumps( {"FE": [{"transformationsData": [{"transformation_label": "novalue"}], "feature": "CRIM", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "3.66", "stddev": "8.64", "min": "0.00632", "max": "88.9762", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "CRIM"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "ZN", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "11.5", "stddev": "23.43", "min": "0.0", "max": "100.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "ZN"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "INDUS", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "11.13", "stddev": "6.9", "min": "0.46", "max": "27.74", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "INDUS"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "CHAS", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.07", "stddev": "0.26", "min": "0.0", "max": "1.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "CHAS"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "NOX", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.55", "stddev": "0.12", "min": "0.385", "max": "0.871", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "NOX"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "RM", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "6.28", "stddev": "0.71", "min": "3.561", "max": "8.78", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "RM"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "AGE", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "68.42", "stddev": "28.27", "min": "2.9", "max": "100.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "AGE"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "DIS", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "3.81", "stddev": "2.11", "min": "1.1296", "max": "12.1265", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "DIS"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "RAD", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "9.64", "stddev": "8.72", "min": "1.0", "max": "24.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "RAD"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "TAX", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "409.62", "stddev": "169.0", "min": "187.0", "max": "711.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "TAX"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "PTRATIO", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "18.43", "stddev": "2.16", "min": "12.6", "max": "22.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "PTRATIO"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "B", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "356.21", "stddev": "91.74", "min": "0.32", "max": "396.9", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "B"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "LSTAT", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "12.7", "stddev": "7.16", "min": "1.73", "max": "37.97", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "LSTAT"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "MEDV", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "22.57", "stddev": "9.24", "min": "5.0", "max": "50.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "MEDV"}]}))

	#transformationPostExecutionHook(autofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run dataprepautomlHooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=functionRegression(autofe, ["CRIM", "ZN", "INDUS", "CHAS", "NOX", "RM", "AGE", "DIS", "RAD", "TAX", "PTRATIO", "B", "LSTAT"], "MEDV")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)


***PREDICT ON TRAINED MODEL***

In [ ]:
import pandas as pd
import numpy as np
import sklearn.metrics

try:
    model=dataAutoML ['model']
    X_test=dataAutoML['X_test']
    y_test=dataAutoML['y_test']
    label=dataAutoML['label']
    columnNames=dataAutoML['columnNames']
    if label in columnNames:
        columnNames.remove(label)
    predicted=label+"_predicted"
    y_predicted=model.predict(X_test)
    df =pd.DataFrame(X_test , columns=columnNames)
    df[label]=y_test
    df[predicted]=y_predicted
    columnNames.insert(0,predicted)
    columnNames.insert(0,label)
    df = df[columnNames]
    R2 = np.round(sklearn.metrics.r2_score(y_test, y_predicted), 1)
    Mean_Squared_Error = np.round(sklearn.metrics.mean_squared_error(y_test, y_predicted), 1)
    Mean_Absolute_Error = np.round(sklearn.metrics.mean_absolute_error(y_test, y_predicted), 1)
    display(" R2 score of Prediction on test data    : %s"%R2)
    display(" Mean Squared Error of Prediction on test data    : %s"%Mean_Squared_Error)
    display(" Mean Absolute Error of Prediction on test data   : %s"%Mean_Absolute_Error)
    display(df.head())
except Exception as ex:
    logging.error(ex)

